<a href="https://colab.research.google.com/github/ninay03/Ai-Powered-Predictive-Maintenance-System-for-Renewable-Energy-Plants/blob/main/Solar_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SOLAR **MODEL**

In [20]:
import numpy as np
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import pickle

# Step 1: Load and Preprocess Data

df = pd.read_csv('/content/solar_sensor_data_100000.csv')

# Drop Timestamp

df = df.drop(columns=['Timestamp'], errors='ignore')

# Check for required columns

target = 'System_Health_Score_%'
if target not in df.columns:
    raise ValueError("Target column is missing in the dataset")

# Remove rows with missing target values

df = df.dropna(subset=[target])

# Split Features and Target

X = df.drop(columns=[target])
y = df[target]

# Convert all columns to numeric (in case of object type)

X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(X.mean())  # Fill NaNs with column mean

# Step 2: Split Data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train Gradient Boosting Model

model = GradientBoostingRegressor(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_samples_split=3,
    min_samples_leaf=2,
    random_state=42,
    validation_fraction=0.2,
    n_iter_no_change=10,
    tol=0.001
)

model.fit(X_train, y_train)

# Step 4: Evaluate Model

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"Training R² Score: {r2_train:.4f}")
print(f"Testing R² Score: {r2_test:.4f}")
print(f"Training Mean Absolute Error: {mae_train:.4f}")
print(f"Testing Mean Absolute Error: {mae_test:.4f}")

# Step 5: Save Model as Pickle File

model_filename = "gradient_boosting_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved successfully as {model_filename}")

# Step 6: Prediction Function

def predict_health_score(input_data):
    """
    Predicts System Health Score based on input data.

    Parameters:
        input_data (dict): Dictionary containing feature values.

    Returns:
        dict: Predicted health score and model accuracy.
    """
    # Load the trained model

    with open(model_filename, 'rb') as file:
        loaded_model = pickle.load(file)

    # Convert input dictionary to DataFrame

    input_df = pd.DataFrame([input_data])

    # Ensure all values are numeric

    input_df = input_df.apply(pd.to_numeric, errors='coerce')
    input_df = input_df.fillna(X.mean())  # Fill NaNs with column mean

    # Make Prediction

    predicted_score = loaded_model.predict(input_df)[0]

    # Calculate prediction accuracy

    accuracy = r2_test * 100  # Convert to percentage

    return {
        "Predicted_System_Health_Score": round(predicted_score, 2),

        "Model_Accuracy": round(accuracy, 2)
    }



Training R² Score: 0.9725
Testing R² Score: 0.9636
Training Mean Absolute Error: 1.3187
Testing Mean Absolute Error: 1.5130
Model saved successfully as gradient_boosting_model.pkl


In [21]:
# Example Usage of Predict Function

example_input = {
    "Temperature_C": 46.79,
    "IR_Hotspot_Temperature_C": 69.11,
    "Current_A": 12.1,
    "Voltage_V": 426.4933,
    "IV_Curve_Efficiency_%": 95.78846,
    "Solar_Irradiance_Wm2": 821.3931,
    "Wind_Speed_mps": 18.7398,
    "Humidity_%": 76.86751,
    "Dust_Level_gm2": 45.7,
    "Power_Output_kW": 172.5934,
    "Harmonic_Distortion_%": 3.649823,
    "Ground_Fault_Current_mA": 89.0,
    "Cooling_Fan_Speed_RPM": 1715.368,
    "Vibration_mms": 1.892182,
    "Tilt_Angle_deg":	2.897056


    # Add more features according to your dataset
}

result = predict_health_score(example_input)
print(json.dumps(result, indent=4))


{
    "Predicted_System_Health_Score": 81.96,
    "Model_Accuracy": 96.36
}


In [22]:
import pickle

# Define the filename
model_filename = "gradient_boosting_model.pkl"

# Save the trained model
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved successfully as {model_filename}")


Model saved successfully as gradient_boosting_model.pkl


In [23]:
# Load the model
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

print("Model loaded successfully")

# Example prediction using the loaded model
sample_input = X_test.iloc[0:1]  # Selecting a sample from test data
predicted_value = loaded_model.predict(sample_input)
print(f"Predicted System Health Score: {predicted_value[0]:.2f}%")


Model loaded successfully
Predicted System Health Score: 102.51%
